<a href="https://colab.research.google.com/github/Malvodio/DeepHistory/blob/main/DeepHistory_Tesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepHistory_Tesis

## Re-entrenamiento del modelo

### Iniciando enviroment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# Clonando repositorio FOM
!git clone https://github.com/Malvodio/DeepHistory

Cloning into 'first-order-model'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 299 (delta 2), reused 2 (delta 0), pack-reused 293
Receiving objects: 100% (299/299), 72.15 MiB | 20.21 MiB/s, done.
Resolving deltas: 100% (153/153), done.
Checking out files: 100% (47/47), done.


Cambiando a repositorio First Order Model

In [ ]:
cd DeepHistory/

/content/drive/MyDrive/first-order-model


In [ ]:
# Instalando requerimientos de repositorio First Order Model
# !pip install -r requirements.txt

In [ ]:
#pip install imgaug==0.2.5

In [ ]:
#pip install scikit-image==0.17.2

In [ ]:
# Verificando dispositivo GPU
import torch

torch.cuda.get_device_name(0)

'Tesla K80'

En este punto, se debe subir el modelo pre-entrenado VOX a la ruta first-order-model/models, además se debe editar el archivo vox-adv-256.yaml y configurar un batch_size de 10

In [ ]:
# Importando librerías necesarias para el re-entrenamiento
from os import path, makedirs
from shutil import copy
import imageio
import numpy as np
import sys
import uuid
import yaml
import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import warnings
from time import gmtime, strftime
from skimage import img_as_ubyte
from ctypes import cdll
from train import train
from modules.generator import OcclusionAwareGenerator
from modules.discriminator import MultiScaleDiscriminator
from modules.keypoint_detector import KPDetector
from frames_dataset import FramesDataset
from modules.util import DownBlock2d
from tqdm import trange
import torch
from torch.utils.data import DataLoader
from logger import Logger
from modules.model import GeneratorFullModel, DiscriminatorFullModel
from torch.optim.lr_scheduler import MultiStepLR
from sync_batchnorm import DataParallelWithCallback
from frames_dataset import DatasetRepeater
from demo import load_checkpoints, make_animation, load_checkpoints_Unet_3

In [ ]:
checkpoint_path = './models/vox-adv-cpk.pth.tar'
config_path = './config/vox-adv-256.yaml'
#generator2, kp_detector2 = load_checkpoints(config_path,checkpoint_path)
#print(kp_detector2)

In [ ]:
checkpoint_path = './models/vox-adv-cpk.pth.tar'
config_path = './config/vox-adv-256.yaml'
generator2, kp_detector2 = load_checkpoints(config_path,checkpoint_path)
print(kp_detector2)

DataParallelWithCallback(
  (module): KPDetector(
    (predictor): Hourglass(
      (encoder): Encoder(
        (down_blocks): ModuleList(
          (0): DownBlock2d(
            (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
          )
          (1): DownBlock2d(
            (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): SynchronizedBatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
          )
          (2): DownBlock2d(
            (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): SynchronizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [ ]:
generator_new, kp_detector_new = load_checkpoints_Unet_3(config_path,checkpoint_path)

TypeError: ignored

In [ ]:
CUDA_VISIBLE_DEVICES=0,1,2,3 python run_unet_3.py --config config/prueba.yaml --device_ids 0,1,2,3

In [ ]:
cd ..

/content/drive/My Drive


In [ ]:
cd first-order-model-mod/

/content/drive/MyDrive/first-order-model-mod


Re-definiendo función para cargar el checkpoint del modelo pre-entrenado

In [ ]:
def load_cpk(checkpoint_path, generator=None, discriminator=None, kp_detector=None,
                 optimizer_generator=None, optimizer_discriminator=None, optimizer_kp_detector=None):
    checkpoint = torch.load(checkpoint_path)
    if generator is not None:
        generator.load_state_dict(checkpoint['generator'])
    if kp_detector is not None:
        kp_detector.load_state_dict(checkpoint['kp_detector'])
    if discriminator is not None:
        try:
            discriminator.load_state_dict(checkpoint['discriminator'])
        except:
            print ('No discriminator in the state-dict. Dicriminator will be randomly initialized')

   # print(kp_detector)
    
    if optimizer_generator is not None:
        optimizer_generator.load_state_dict(checkpoint['optimizer_generator'])
    if optimizer_discriminator is not None:
        try:
            optimizer_discriminator.load_state_dict(checkpoint['optimizer_discriminator'])
        except RuntimeError as e:
            print ('No discriminator optimizer in the state-dict. Optimizer will be not initialized')
    if optimizer_kp_detector is not None:
        optimizer_kp_detector.load_state_dict(checkpoint['optimizer_kp_detector'])

    return checkpoint['epoch']

Estableciendo parámetros iniciales y re-entrenando modelo

In [ ]:
warnings.filterwarnings("ignore")

config = './config/vox-adv-256.yaml'
device_ids = [0]
#checkpoint = './models/vox-adv-cpk.pth.tar'
checkpoint = None
log_dir = './logs'

if __name__ == "__main__":
    
    with open(config) as f:
        config = yaml.load(f)
        
    generator = OcclusionAwareGenerator(**config['model_params']['generator_params'], **config['model_params']['common_params'])

    if torch.cuda.is_available():
        generator.to(device_ids[0])

    discriminator = MultiScaleDiscriminator(**config['model_params']['discriminator_params'], **config['model_params']['common_params'])

    if torch.cuda.is_available():
        discriminator.to(device_ids[0])

    kp_detector = KPDetector(**config['model_params']['kp_detector_params'], **config['model_params']['common_params'])

    if torch.cuda.is_available():
        kp_detector.to(device_ids[0])
            
    dataset = FramesDataset(is_train=1, **config['dataset_params'])


    print("Training...")

    train_params = config['train_params']

    optimizer_generator = torch.optim.Adam(generator.parameters(), lr=train_params['lr_generator'], betas=(0.5, 0.999))
    optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=train_params['lr_discriminator'], betas=(0.5, 0.999))
    optimizer_kp_detector = torch.optim.Adam(kp_detector.parameters(), lr=train_params['lr_kp_detector'], betas=(0.5, 0.999))

    if checkpoint is not None:
        start_epoch = load_cpk(checkpoint, generator, discriminator, kp_detector, optimizer_generator, optimizer_discriminator, None if train_params['lr_kp_detector'] == 0 else optimizer_kp_detector)
    else:
        start_epoch = 0
    
    # Changes
    #kp_detector.predictor.encoder.down_blocks[0].conv.kernel_size = (5, 4)
    #kp_detector.predictor.encoder.down_blocks[1].conv.kernel_size = (5, 4)
    #kp_detector.predictor.encoder.down_blocks[2].conv.kernel_size = (5, 4)
    #kp_detector.predictor.encoder.down_blocks[3].conv.kernel_size = (5, 4)
    #kp_detector.predictor.encoder.down_blocks[4].conv.kernel_size = (4, 4)
    #print(kp_detector.predictor.encoder)
    # End Changes

    scheduler_generator = MultiStepLR(optimizer_generator, train_params['epoch_milestones'], gamma=0.1, last_epoch=start_epoch - 1)

    scheduler_discriminator = MultiStepLR(optimizer_discriminator, train_params['epoch_milestones'], gamma=0.1, last_epoch=start_epoch - 1)

    scheduler_kp_detector = MultiStepLR(optimizer_kp_detector, train_params['epoch_milestones'], gamma=0.1, last_epoch=-1 + start_epoch * (train_params['lr_kp_detector'] != 0))

    if 'num_repeats' in train_params or train_params['num_repeats'] != 1:
        dataset = DatasetRepeater(dataset, train_params['num_repeats'])

    dataloader = DataLoader(dataset, batch_size=train_params['batch_size'], shuffle=True, num_workers=6, drop_last=True)

    generator_full = GeneratorFullModel(kp_detector, generator, discriminator, train_params)
    discriminator_full = DiscriminatorFullModel(kp_detector, generator, discriminator, train_params)

    if torch.cuda.is_available():
        generator_full = DataParallelWithCallback(generator_full, device_ids=device_ids)
        discriminator_full = DataParallelWithCallback(discriminator_full, device_ids=device_ids)

    with Logger(log_dir=log_dir, visualizer_params=config['visualizer_params'], checkpoint_freq=train_params['checkpoint_freq']) as logger:
        for epoch in trange(start_epoch, train_params['num_epochs']):
            for x in dataloader:
                #print(dataloader)
                losses_generator, generated = generator_full(x)

                loss_values = [val.mean() for val in losses_generator.values()]
                loss = sum(loss_values)

                loss.backward()
                optimizer_generator.step()
                optimizer_generator.zero_grad()
                optimizer_kp_detector.step()
                optimizer_kp_detector.zero_grad()

                if train_params['loss_weights']['generator_gan'] != 0:
                    optimizer_discriminator.zero_grad()
                    losses_discriminator = discriminator_full(x, generated)
                    loss_values = [val.mean() for val in losses_discriminator.values()]
                    loss = sum(loss_values)

                    loss.backward()
                    optimizer_discriminator.step()
                    optimizer_discriminator.zero_grad()
                else:
                    losses_discriminator = {}

                losses_generator.update(losses_discriminator)
                losses = {key: value.mean().detach().data.cpu().numpy() for key, value in losses_generator.items()}
                logger.log_iter(losses=losses)

            scheduler_generator.step()
            scheduler_discriminator.step()
            scheduler_kp_detector.step()
            
            logger.log_epoch(epoch, {'generator': generator, 'discriminator': discriminator, 'kp_detector': kp_detector, 'optimizer_generator': optimizer_generator, 'optimizer_discriminator': optimizer_discriminator, 'optimizer_kp_detector': optimizer_kp_detector}, inp=x, out=generated)

Use predefined train-test split.
Training...


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

100%|██████████| 10/10 [14:45<00:00, 88.59s/it]


In [ ]:
checkpoint_path = './logs/00000009-checkpoint.pth.tar'
config_path = './config/vox-adv-256.yaml'
generator1, kp_detector1 = load_checkpoints(config_path,checkpoint_path)
print(kp_detector1.module.predictor)

Hourglass(
  (encoder): Encoder(
    (down_blocks): ModuleList(
      (0): DownBlock2d(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
      )
      (1): DownBlock2d(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): SynchronizedBatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
      )
      (2): DownBlock2d(
        (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): SynchronizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (pool): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
      )
      (3): DownBlock2d(
        (conv): Conv2d(256

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python run.py --config config/vox-adv-256.yaml --device_ids 0

run.py:40: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
Use predefined train-test split.
Training...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0% 0/150 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "run.py", line 81, in <module>
    train(config, generator, discriminator, kp_detector, opt.checkpoint, log_dir, dataset, opt.device_ids)
  File "/content/first-order-model/train.py", line 50, in train
    for x in dataloader:
  Fil

In [ ]:
!rm -rf video-preprocessing/vox-png

In [ ]:
!rmdir 'data/vox-png/train/id11181#cu11S_2dNyA#007258#007702.mp4/.ipynb_checkpoints'